# Jira SQL to generate the file
project = "BPD" AND labels = Release_Notes AND fixversion = 12-13-2024 ORDER BY key DESC, created DESC

In [ ]:
# Import Necessary Libraries
import os
import yaml
import pandas as pd
from datetime import datetime
import openai

In [ ]:
# Define Variables
PROMPTS_DIR = "prompts"
CREATE_RELEASE_NOTE_PROMPT_FILE = os.path.join(PROMPTS_DIR, "create_release_notes.yaml")
CATEGORIZE_RELEASE_NOTE_PROMPT_FILE = os.path.join(PROMPTS_DIR, "categorize_release_notes.yaml")

# Path to the CSV file containing Jira issues
CSV_FILE_PATH = "path/to/your/jira_issues.csv"  # Update this path

# OpenAI API Key
SECRETS_FILE = "supporting_files/secrets.yaml"
with open(SECRETS_FILE, "r") as secrets_file:
    secrets = yaml.safe_load(secrets_file)
openai.api_key = secrets["openai_api_key"]

In [ ]:
# Load YAML Prompts
def load_yaml_prompt(file_path):
    """Load a YAML prompt from a specified file."""
    with open(file_path, "r") as file:
        return yaml.safe_load(file)

create_release_note_prompt = load_yaml_prompt(CREATE_RELEASE_NOTE_PROMPT_FILE)
categorize_release_notes_prompt = load_yaml_prompt(CATEGORIZE_RELEASE_NOTE_PROMPT_FILE)

In [ ]:
# Utility Functions
def get_day_suffix(day):
    """Returns the appropriate suffix for a given day."""
    if 4 <= day <= 20 or 24 <= day <= 30:
        return "th"
    else:
        return ["st", "nd", "rd"][day % 10 - 1]

def get_formatted_date():
    """Returns today's date formatted with day of the week and suffix."""
    today = datetime.today()
    day = today.day
    suffix = get_day_suffix(day)
    month = today.strftime("%B")
    year = today.year
    day_of_week = today.strftime("%A")
    return f"{day_of_week}, {month} {day}{suffix}, {year}"

In [ ]:
# Create Release Note
def create_release_note_for_story(title, description, labels):
    """Generates a single-sentence release note using OpenAI."""
    prompt = create_release_note_prompt["full_prompt"]
    filled_prompt = prompt.replace("{{title}}", title).replace("{{description}}", description).replace("{{labels}}", labels or "No labels provided")
    
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": filled_prompt}
    ]
    
    response = openai.ChatCompletion.create(
        model=create_release_note_prompt["recommended_model"]["model"],
        messages=messages,
        max_tokens=500
    )
    return response["choices"][0]["message"]["content"]

In [ ]:
# Categorize Release Notes
def categorize_release_notes(release_notes, categories):
    """Categorizes release notes using OpenAI."""
    prompt = categorize_release_notes_prompt["full_prompt"]
    filled_prompt = prompt.replace("{{categories}}", ", ".join(categories)).replace("{{release_notes}}", release_notes)
    
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": filled_prompt}
    ]
    
    response = openai.ChatCompletion.create(
        model=categorize_release_notes_prompt["recommended_model"]["model"],
        messages=messages,
        max_tokens=4096
    )
    return response["choices"][0]["message"]["content"]

In [ ]:
# Generate Release Notes
def generate_release_notes_from_csv(csv_file_path, categories):
    """Processes Jira issues from a CSV file and generates categorized release notes."""
    issues_df = pd.read_csv(csv_file_path)
    release_notes = ""
    
    for _, row in issues_df.iterrows():
        title = row.get("Summary", "No Title Provided")
        description = row.get("Description", "No Description Provided")
        labels = row.get("Labels", "No Labels Provided")
        release_note = create_release_note_for_story(title, description, labels)
        release_notes += f"<li>{release_note}</li>\n"
    
    categorized_notes = categorize_release_notes(release_notes, categories)
    formatted_date = get_formatted_date()
    
    return f"""
<p>
  Our latest product release took effect on <strong>{formatted_date}</strong>.
  This post may differ from the release notes sent via email.
</p>
<p>
  For complete documentation, visit the 
  <a href="https://support.brightidea.com/hc/en-us/sections/200825397-Product-Release-Notes">Product Release Notes</a> forum in the Support Portal.
</p>
{categorized_notes}
"""

In [ ]:
# Cell to Run
CATEGORIES = ["General bug fixes and enhancements", "Project Room", "Whiteboard", "Hackathon"]

release_notes_html = generate_release_notes_from_csv(CSV_FILE_PATH, CATEGORIES)
print(release_notes_html)